# Phase Estimation from the box

This lab is based on `PhaseEstimation` class from `qiskit.circuit.library`.

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, BasicAer, execute
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import PhaseEstimation
import matplotlib.pyplot as plt
import numpy as np
from cmath import phase, pi, e

### Let's prepare complex gate combination with non-trivial phase.

We will do this in a matrix, and in a circuit form.

In [ ]:
H = np.array([[1, 1], [1, -1]], dtype=complex) * (.5 ** .5)
CX = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])
U = CX @ np.kron(H, H)

evals, evecs = np.linalg.eig(U)
for val, vec in zip(evals, evecs.T):
    print(f"{np.round(val, 3):15} -> {np.round(vec, 3)}")

print()

# we will run the method for this particular vector.
# you may change the value to see how the method works
# with other eigenstates
WHICH_VECTOR = 3

vector = evecs[:, WHICH_VECTOR]
print("Eigenvalue:  ", np.round(evals[WHICH_VECTOR], 3))
print("Eigenvector: ", np.round_(vector, 4))
print()

# lambda = e^(i * phi)
p = phase(evals[WHICH_VECTOR])
positive_p = p % (2 * pi)
print(f"φ = {p / pi:.3f} π (or {positive_p / pi:.3f} π)")

### Here is the prepared circuit for QPE

[Docs are here](https://qiskit.org/documentation/stubs/qiskit.circuit.library.PhaseEstimation.html#).

In [ ]:
strange_circuit = QuantumCircuit(2)
strange_circuit.h([0, 1])
strange_circuit.cx(1, 0)
strange_circuit.draw()

In [ ]:
# number of qubits for measurements.
# changing this number to a bigger or a smaller value
# will lead to a smaller or a bigger error correspondingly
PRECISION = 9

qpe = PhaseEstimation(
    PRECISION,        # computation precision
    strange_circuit   # the matrix
)
# display(qpe.draw('mpl'))
display(qpe.decompose().draw('mpl'))

QPE is a circuit. Thus, you may append it to your bigger circuit.

**TODO:**
1. Initialize last 2 qubits with an eigenvector `vector`.
2. Append QPE circuit.
3. Measure precision qubits. **NB: they will be given in reverse order, thus you will obseve a reversed number.**

In [ ]:
GATESIZE = 2
qc = QuantumCircuit(PRECISION + GATESIZE, PRECISION)

qc.initialize(vector, range(PRECISION, PRECISION + GATESIZE))  # 1
qc.append(qpe, range(PRECISION + GATESIZE))                    # 2
qc.measure(range(PRECISION), range(PRECISION))                 # 3

qc.draw('mpl')

In [ ]:
shots = 1000
job = execute(qc, BasicAer.get_backend('qasm_simulator'), shots=shots)
counts = job.result().get_counts(qc)
plot_histogram(counts)

Lasts steps are here.

I extracted the most probable measurement for you.
TODO:
1. Convert `measurement` into an `angle`, rememering, that $2\pi$ is split into $2^{PRECISION}$ sectors.
2. Convert an `angle` to an `eigenvalue`. Angle is just an [argument of a complex number](https://en.wikipedia.org/wiki/Argument_(complex_analysis)).

In [ ]:
most_frequent = sorted(counts.items(), key=lambda x: x[1])[-1][0]
# reversed bit order!
measurement = int(most_frequent[::-1], base=2)
print(f"Measured ({most_frequent}): {measurement} / {2 ** PRECISION}")

angle = measurement * (2 * pi) / (2 ** PRECISION)
print(f"Angle: {angle / pi} π")
print(f"Angle error: { abs(positive_p - angle) / abs(angle) * 100:.2f}%")

eigenvalue = e ** (1j * angle)
print("Computed eigenvalue: ", eigenvalue)
print("Expected eigenvalue: ", evals[WHICH_VECTOR])